In [29]:
using Pkg

Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("BenchmarkTools")
Pkg.add("Distances")
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
struct Delivery
    x::Float64
    y::Float64
end;
function random_delivery()
    x = rand(0.0:0.1:100.0)
    y = rand(0.0:0.1:100.0)
    return Delivery(x, y)
end;

struct TSProblem
    deliveries::Array{Delivery}
end;

function random_instance(n_deliveries)
    deliveries = [random_delivery() for _=1:n_deliveries]
    problem = TSProblem(deliveries)
    return problem
end;

function dist(del1::Delivery, del2::Delivery)
    return sqrt((del1.x - del2.x)^2 + (del1.y - del2.y)^2)
end
function calc_travelmatrix(deliveries::Array{Delivery})
    tm = zeros(Float64, length(deliveries), length(deliveries))
    for i = 1:length(deliveries)
        for j = 1:length(deliveries)
            tm[i, j] = dist(deliveries[i], deliveries[j])
        end
    end
    return tm
end


calc_travelmatrix (generic function with 1 method)

In [43]:
using JuMP
import GLPK
using BenchmarkTools
using Plots

function has_sub_tours(m, routes)
    N = size(routes)[1]
    x_val = JuMP.value.(routes)
    
    # find cycle
    cycle_idx = Int[]
    push!(cycle_idx, 1)
    while true
        v, idx = findmax(x_val[cycle_idx[end],1:N])
        if idx == cycle_idx[1]
            break
        else
            push!(cycle_idx,idx)
        end
    end
    if length(cycle_idx) < N
        @constraint(m, sum(routes[cycle_idx,cycle_idx]) <= length(cycle_idx)-1)
        return true
    end
    return false
end
function display_solution(problem, route)
    x_pos = [c.x for c in problem.deliveries]
    y_pos = [c.y for c in problem.deliveries]
    plot_result = scatter(x_pos, y_pos, shape = :circle, markersize = 6)
    for i in 1:length(problem.deliveries)
        for j in 1:length(problem.deliveries)
            val = route[i, j]
            if val > 0
                del1 = problem.deliveries[i]
                del2 = problem.deliveries[j]
                plot!([del1.x, del2.x], [del1.y, del2.y], legend = false)
            end
        end
    end
    return plot_result
end;

function solve_tsp(deliveries::Int64, solver)
    problem=random_instance(deliveries)
    travelmatrix = calc_travelmatrix(problem.deliveries)
    model = Model(solver)
    # route is an adjence matrix representing a route traveled
    route=@variable(model, route[1:length(problem.deliveries), 1:length(problem.deliveries)], Bin)

    # ensure all events are planned
    @constraint(model, [i = 1:length(problem.deliveries)], sum(route[i, :]) == 1.0)
    # ensure there is just one route
    @constraint(model, [c = 1:length(problem.deliveries)], sum(route[:, c]) == 1.0)
    # disallow traveling to itself
    @constraint(model, [j = 1:length(problem.deliveries)], route[j, j] == 0)

    traveltime = travelmatrix.* route 
    @objective(model, Min, sum(traveltime))
    optimize!(model)
    route_val = JuMP.value.(route)
    while has_sub_tours(model, route)
        optimize!(model)
        route_val = JuMP.value.(route)
    end
    #return display_solution(problem, route_val)
end

@benchmark solve_tsp(20, GLPK.Optimizer)



BenchmarkTools.Trial: 21 samples with 1 evaluation.
 Range (min … max):   30.277 ms … 765.485 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     163.423 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   257.711 ms ± 234.647 ms  ┊ GC (mean ± σ):  0.23% ± 0.35%

  ▃     █                                   ▃                    
  █▇▁▇▇▇█▁▇▁▁▇▇▇▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇█▁▁▁▁▇▁▁▁▇▁▁▁▁▁▁▁▁▇ ▁
  30.3 ms          Histogram: frequency by time          765 ms <

 Memory estimate: 5.00 MiB, allocs estimate: 169269.